In [1]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
Mon Dec 12 15:06:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    28W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |      

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Colab Notebooks/DD2360 Applied GPU Programming KTH/Project'
!ls

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/DD2360 Applied GPU Programming KTH/Project
bin	  include  Makefile	       README.md  tmp.txt
datasets  knn.cu   Parallel_knn.ipynb  src


In [7]:
!make clean
#!make all
!make knn
!chmod 755 ./bin/knn

rm -rf  src/terminal_user_input.o src/test.o src/knn.o
rm -rf bin/knn
rm -rf bin/test
[ -d src ] || mkdir src
nvcc -std=c++11 -I./include -O0 -g -G -Xcompiler -Wall -lm -DNDEBUG src/terminal_user_input.cpp -c -o src/terminal_user_input.o
nvcc -I./include -arch=sm_75 -std=c++11 -O0 -g -G -Xcompiler -Wall --compiler-bindir=g++ -lm -DNDEBUG src/knn.cu -c -o src/knn.o
nvcc -I./include -arch=sm_75 -std=c++11 -O0 -g -G -Xcompiler -Wall --compiler-bindir=g++ -lm -DNDEBUG src/terminal_user_input.o src/knn.o -o bin/knn


**Debugging:**
- https://stackoverflow.com/questions/70626304/how-to-debug-a-cuda-google-colab-notebook
- https://stackoverflow.com/questions/14038589/what-is-the-canonical-way-to-check-for-errors-using-the-cuda-runtime-api
- https://www.olcf.ornl.gov/calendar/cuda-debugging/
- https://colab.research.google.com/drive/1GJOfTp56OeQRdE4u2_S7pUNRcJb4ik9X?usp=sharing
- https://docs.nvidia.com/cuda/cuda-gdb/index.htm
    - (cuda-gdb) break knn.cu:238
    - (cuda-gdb) break my_function
    - (cuda-gdb) next
    - (cuda-gdb) run
    - (cuda-gdb) print &array <br/>
      $1 = (@shared int (*)[0]) 0x20
    - (cuda-gdb) print array[0]@4 <br/>
      $2 = {0, 128, 64, 192}
    - (cuda-gdb) info cuda blocks
    - (cuda-gdb) info cuda threads
    - (cuda-gdb) up
    - (cuda-gdb) where

datasets/iris_dataset/iris.data.csv<br/>
print datapoints->points[0].dimension[0]

In [12]:
#!./bin/knn
#!printf "set cuda memcheck on\nset cuda api_failures stop\ncatch throw\nr\nbt\ninfo locals\nthread 1\nbt\n" > ./tmp.txt
#!cuda-gdb -batch -x tmp.txt --args ./bin/knn
#!cuda-gdb ./bin/knn
!compute-sanitizer --tool memcheck ./bin/knn

========= COMPUTE-SANITIZER
Filename: datasets/iris_dataset/iris.data.csv
datapoints->points[0].dimension[0] = 4.700000
========= Program hit invalid argument (error 1) on CUDA API call to cudaMemcpy.
=========     Saved host backtrace up to driver entry point at error
=========     Host Frame: [0x7f0683fbd373]
=========                in /usr/lib64-nvidia/libcuda.so.1
=========     Host Frame: [0x5592cf6572fd]
=========                in /content/drive/MyDrive/Colab Notebooks/DD2360 Applied GPU Programming KTH/Project/./bin/knn
=========     Host Frame: [0x5592cf608d80]
=========                in /content/drive/MyDrive/Colab Notebooks/DD2360 Applied GPU Programming KTH/Project/./bin/knn
=========     Host Frame: [0x5592cf60a267]
=========                in /content/drive/MyDrive/Colab Notebooks/DD2360 Applied GPU Programming KTH/Project/./bin/knn
=========     Host Frame: [0x5592cf60a404]
=========                in /content/drive/MyDrive/Colab Notebooks/DD2360 Applied GPU Programmin